# Test analytical derivatives of $p_v\left(c\right)$ computed by private methods of `Polyclonal`

Set up a toy example:

In [1]:
import numpy

import pandas as pd

from polyclonal import Polyclonal

activity_wt_df = pd.DataFrame({'epitope':  ['e1', 'e2'],
                               'activity': [ 2.0,  1.0]})

mut_escape_df = pd.DataFrame({
       'mutation': ['M1C', 'M1C', 'G2A', 'G2A', 'A4K', 'A4K', 'A4L', 'A4L'],
       'epitope':  [ 'e1',  'e2',  'e1',  'e2',  'e1',  'e2',  'e1',  'e2'],
       'escape':   [  2.0,   0.0,   3.0,   0.0,  0.0,    2.5,   0.0,   1.5],
       })

polyclonal_sim = Polyclonal(activity_wt_df=activity_wt_df,
                            mut_escape_df=mut_escape_df)

variants_df = pd.DataFrame.from_records(
         [('AA', ''),
          ('AC', 'M1C'),
          ('AG', 'G2A'),
          ('AT', 'A4K'),
          ('TA', 'A4L'),
          ('CA', 'M1C G2A'),
          ('CG', 'M1C A4K'),
          ('CC', 'G2A A4K'),
          ('TC', 'G2A A4L'),
          ('CT', 'M1C G2A A4K'),
          ('TG', 'M1C G2A A4L'),
          ('GA', 'M1C'),
          ],
         columns=['barcode', 'aa_substitutions'])

escape_probs = polyclonal_sim.prob_escape(variants_df=variants_df,
                                          concentrations=[1.0, 2.0, 4.0])

data_to_fit = (
         escape_probs
         .rename(columns={'predicted_prob_escape': 'prob_escape'})
         )

polyclonal_data = Polyclonal(data_to_fit=data_to_fit,
                             activity_wt_df=activity_wt_df,
                             site_escape_df=pd.DataFrame.from_records(
                                    [('e1', 1, 1.0), ('e1', 4, 0.0),
                                     ('e2', 1, 0.0), ('e2', 4, 2.0)],
                                    columns=['epitope', 'site', 'escape']),
                             data_mut_escape_overlap='fill_to_data',
                             )

Now compute `p_vc` and `dpvc_dparams`:

In [2]:
p_vc, dpvc_da, dpvc_dbeta = polyclonal_data._compute_pv(
                                params=polyclonal_data._params,
                                bmap=polyclonal_data._binarymaps,
                                cs=polyclonal_data._cs,
                                calc_grad=True,
                                )

<class 'scipy.sparse.coo.coo_matrix'>


Get numerical estimates of gradient and check they are close to analytical estimates:

In [7]:
eps = 1e-8
for iparam in range(len(polyclonal_data._params)):
    eps_vec = numpy.zeros(len(polyclonal_data._params))
    eps_vec[iparam] = eps
    p_vc_eps = polyclonal_data._compute_pv(
                        params=polyclonal_data._params + eps_vec,
                        bmap=polyclonal_data._binarymaps,
                        cs=polyclonal_data._cs,
                        calc_grad=False,
                        )
    numerical_grad = (p_vc_eps - p_vc) / eps
    if iparam < 2:
        analytical_grad = dpvc_da[iparam]
    else:
        analytical_grad = dpvc_dbeta.toarray()[iparam - 2]
    diff = numpy.sqrt(((numerical_grad - analytical_grad)**2).sum())
    mag = numpy.sqrt((analytical_grad**2).sum())
    if (diff > 1e-7) or (diff / mag > 1e-7):
        raise ValueError(f"{iparam=}, {diff=}, {mag=}\n"
                         f"{analytical_grad=}\n{numerical_grad=}")

In [4]:
print("numerical:\n", grad[: 2])

print('\nanalytical:\n', dpvc_da)

NameError: name 'grad' is not defined

In [ ]:
polyclonal_data.mut_escape_df

In [ ]:
polyclonal_data._a_beta_from_params(polyclonal_data._params)

In [ ]:
polyclonal_data._params[2:]